In [1]:
import scipy
import keras_cv
import keras
import matplotlib.pyplot as plt
from keras import ops
import numpy as np
import math
from PIL import Image

# Enable mixed precision
# (only do this if you have a recent NVIDIA GPU)
keras.mixed_precision.set_global_policy("mixed_float16")

# Instantiate the Stable Diffusion model
model = keras_cv.models.StableDiffusion(jit_compile=True)

By using this model checkpoint, you acknowledge that its usage is subject to the terms of the CreativeML Open RAIL-M license at https://raw.githubusercontent.com/CompVis/stable-diffusion/main/LICENSE


In [3]:
prompt_1 = "A watercolor painting of a Golden Retriever at the beach"
prompt_2 = "A still life DSLR photo of a bowl of fruit"
interpolation_steps = 5

encoding_1 = ops.squeeze(model.encode_text(prompt_1))
encoding_2 = ops.squeeze(model.encode_text(prompt_2))

interpolated_encodings = ops.linspace(encoding_1, encoding_2, interpolation_steps)

# Show the size of the latent manifold
print(f"Encoding shape: {encoding_1.shape}")

1356917/1356917 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step

492466864/492466864 ━━━━━━━━━━━━━━━━━━━━ 44s 0us/step
Encoding shape: (77, 768)


In [55]:
def word_similarity(word1: str, word2: str, encoding_model) -> float:
    encoding1 = encoding_model.encode_text(word1)
    encoding2 = encoding_model.encode_text(word2)
    distance = scipy.spatial.distance.euclidean(encoding1.flatten(), encoding2.flatten())

    return distance

In [66]:
word1 = "toyota"
word2 = "house"
print(word_similarity(word1, word2, model))

232.1
